In [1]:
from __future__ import annotations
from dataclasses import dataclass
from typing import Dict, Union, Optional, Iterable
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Load Results from all strategies

## Plot 1: Signal Overlay on Price Chart

## Plot 2: Holdings, Cash and Total Assets Over Time

## Plot 5: Compare Cumulative PnL Across Strategies

## Reflection on Signal Behavior, Execution Constraints, and Performance